# 2CSSID-TP03. Naive Bayes

---

Dans ce TP, nous allons traiter Naive Bayes. C'est un modèle génératif.

- **Binome 01** : Tirichine Mohammed
- **Binome 02** : Bourzag Mohamed Chakib
- **Groupe** : Il n'y a qu'un groupe => info inutile

In [1]:
import matplotlib
import numpy             as np
import pandas            as pd 
import matplotlib.pyplot as plt 
%matplotlib inline

np        .__version__ , \
pd        .__version__ , \
matplotlib.__version__

('1.26.1', '2.1.2', '3.8.1')

In [2]:
from typing import Tuple, List, Dict

**RAPPEL**

Tout le monde connait le théorème de Bayes pour calculer la probabilité conditionnelle d'un évennement $A$ sachant un autre $B$: 
$$ P(A|B) = \frac{P(A)P(B|A)}{P(B)}$$

Pour appliquer ce théorème sur un problème d'appentissage automatique, l'idée est simple ; Etant donné une caractéristique $f$ et la sortie $y$ qui peut avoir la classe $c$ : 
- Remplacer $A$ par $y=c$
- Remplacer $B$ par $f$ 
On aura l'équation : 
$$ P(y=c|f) = \frac{P(y=c)P(f|y=c)}{P(f)}$$

On appelle : 
- $P(y=c|f)$ postérieure 
- $P(y=c)$ antérieure
- $P(f|y=c)$ vraisemblance
- $P(f)$ évidence 

Ici, on estime la probablité d'une classe $c$ sachant une caractéristique $f$ en utilisant des données d'entrainement. Maintenant, on veut estimer la probabilité d'une classe $c$ sachant un vecteur de caractéristiques $\overrightarrow{f} = \{f_1, ..., f_L\}$ : 
$$ P(y=c|\overrightarrow{f}) = \frac{P(y=c)P(\overrightarrow{f}|y=c)}{P(\overrightarrow{f})}$$

Etant donnée plusieurs classes $c_j$, la classe choisie $\hat{c}$ est celle avec la probabilité maximale 
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k|\overrightarrow{f})$$
$$\hat{c} = \arg\max\limits_{c_k} \frac{P(y=c_k)P(\overrightarrow{f}|y=c_k)}{P(\overrightarrow{f})}$$
On supprime l'évidence pour cacher le crime : $P(\overrightarrow{f})$ ne dépend pas de $c_k$ et elle est postive, donc ça ne va pas affecter la fonction $\max$.
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k)P(\overrightarrow{f}|y=c_k)$$

Pour calculer $P(\overrightarrow{f}|y=c_k)$, on va utiliser une properiété naïve (d'où vient le nom Naive Bayes) : on suppose l'indépendence conditionnelle entre les caractéristiques $f_j$. 
$$\hat{c} = \arg\max\limits_{c_k} P(y=c_k) \prod\limits_{f_j \in \overrightarrow{f}} P(f_j|y=c_k)$$

Pour éviter la disparition de la probabilité (multiplication et représentation de virgule flottante sur machine), on transforme vers l'espace logarithme.
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$


## I. Réalisation des algorithmes

Pour estimer la vraisemblance, il existe plusieurs modèles (lois):
- **Loi multinomiale :** pour les caracétristiques nominales
- **Loi de Bernoulli :** lorsqu'on est interressé par l'apparence d'une caractéristique ou non (binaire)
- **Loi normale :** pour les caractéristiques numériques

Dans ce TP, nous allons implémenter Naive Bayes pour les caractéristiques nominales (loi multinomiale). 
Dans notre modèle, nous voulons stocker les statistiques et pas les probabilités. 
L'intérêt est de faciliter la mise à jours des statistiques (si par exemple, nous avons un autre dataset et nous voulons enrichir le modèle ; dans e cas, il suffit d'ajouter les statistiques du nouveau dataset)

Ici, nous allons utiliser le dataset "jouer" (utilisé dans la plupart des cours) contenant des caractéristiques nominales.

In [3]:
jouer   = pd.read_csv('data/jouer.csv')

X_jouer = jouer.iloc[:, :-1].values # Premières colonnes 
Y_jouer = jouer.iloc[:,  -1].values # Dernière colonne 

# Afficher le dataset "jouer"
jouer

,temps,temperature,humidite,vent,jouer
0,ensoleile,chaude,haute,non,non
1,ensoleile,chaude,haute,oui,non
2,nuageux,chaude,haute,non,oui
3,pluvieux,douce,haute,non,oui
4,pluvieux,fraiche,normale,non,oui
5,pluvieux,fraiche,normale,oui,non
6,nuageux,fraiche,normale,oui,oui
7,ensoleile,douce,haute,non,non
8,ensoleile,fraiche,normale,non,oui
9,pluvieux,douce,normale,non,oui


### I.1. Entraînement de la probabilité antérieure

Etant donné le vecteur de sortie $Y$, la probabilité de chaque classe (différentes valeurs de $Y$) est calulée comme :

$$p(c_k) = \frac{|\{y / y \in Y \text{ et } y = c_k\}|}{|Y|}$$


La fonction doit récupérer des statistiques afin de pouvoir calculer la probabilité antérieure de chaque classe. Donc, elle doit retourner  :
- Un vecteur contenant les noms des classes
- Un vecteur contenant les nombres d'occurrences de chaque classe dans le premier vecteur

In [61]:
# TODO: Stastistiques sur la probabilité antérieure
def stat_anterieure(Y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]: 
    cls, counts = np.unique(Y, return_counts=True)
    
    return cls, counts.astype(int)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# (array(['non', 'oui'], dtype=object), array([5, 9]))
#---------------------------------------------------------------------

stat_anterieure(Y_jouer)

(array(['non', 'oui'], dtype=object), array([5, 9]))

### I.2. Entraînement de la probabilité de vraissemblance (loi multinomiale)

Notre modèle doit garder le nombre des différentes valeurs d'une caractéristique $A$ et le nombre de ces valeurs dans chaque classe.
Donc, étant donné un vecteur d'une caractéristique $A= X[:,j]$, un autre des $Y$ et un $C$ contenant la liste des classes, la fonction d'entraînement doit retourner : 
- $V$ : un vecteur contenant les différentes catégories de $A$ (c'est déjà fait)
- Une matrice contenant le nombre d'occurrences de chaque catégorie de $V$ dans chaque classe  : 
   - Les lignes représentent les catégories $v \in V$ de la caréctéristique $A$
   - Les colonnes représentent les classes $c \in C$ de $Y$


In [65]:
# TODO: Statistiques de vraissemblance (une seule caractéristique)
def stat_vraissemblance_1(A: np.ndarray, 
                          Y: np.ndarray, 
                          C: np.ndarray
                         ) -> Tuple[np.ndarray, np.ndarray]: 
    V = np.unique(A) # Catégories de la caractéristique A
    freq = np.zeros((len(V), len(C)))
    d = np.concatenate((
        np.array([np.where(V == ele) for ele in A]).reshape(-1, 1),
        np.array([np.where(C == ele) for ele in Y]).reshape(-1, 1)
    ), axis=1)

    inds, counts = np.unique(d, axis=0, return_counts=True)
    for i in range(len(inds)) :
        freq[inds[i][0], inds[i][1]] = counts[i]
    
    return V, freq.astype(int)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# (array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object),
#  array([[3, 2],
#         [0, 4],
#         [2, 3]]))
#---------------------------------------------------------------------
C_t = np.array(['non', 'oui'])
stat_vraissemblance_1(X_jouer[:, 0], Y_jouer, C_t)

(array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object),
 array([[3, 2],
        [0, 4],
        [2, 3]]))

### I.3. Entraînement loi multinomiale

**Rien à programmer ici**

Notre modèle ($\theta_{X, C}$) doit garder des statistiques sur les classes et aussi sur chaque catégorie de chaque caractéristique. Pour ce faire, nous allons représenter $\theta$ comme un vecteur : 
- $\theta[N+1]$ est un vecteur de $N$ éléments représentant des statistiques sur chaque caractéristique $j$, plus un élément (le dernier) pour les statistiques sur les classes.
- Chaque élément est un dictionnaire (HashMap en Java)
- Un élément des caractéristiques contient deux clés : 
    - **val** : pour récupérer la liste des noms des catégories de la caractéristique
    - **freq**: pour récupérer une matrice représentant la fréquence de chaque caractéristique dans chaque classe
- Un élément des classes contient deux clés : 
    - **cls** : pour récupérer la liste des noms des classes
    - **freq**: pour récupérer la liste des fréquences de chaque classe

In [63]:
# La fonction qui entraine Théta sur plusieurs caractéristiques
# Rien à programmer ici
# Notre théta est une liste des dictionnaires;
# chaque dictionnaire contient la liste des catégories et la matrice des fréquences dela caractéristique respective à la colonne de X
# On ajoute les statistiques antérieures des classes à la fin de résultat
def entrainer_multi(X: np.ndarray, 
                    Y: np.ndarray
                   ) -> np.ndarray: 
    
    Theta   = []
    
    stats_c = {}
    stats_c['cls'], stats_c['freq'] =  stat_anterieure(Y)
    
    for j in range(X.shape[1]): 
        stats = {}
        stats['val'], stats['freq'] =  stat_vraissemblance_1(X[:, j], Y, stats_c['cls'])
        Theta.append(stats)
    
    Theta.append(stats_c)
    return Theta


#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# [{'val': array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object),
#   'freq': array([[3, 2],
#          [0, 4],
#          [2, 3]])},
#  {'val': array(['chaude', 'douce', 'fraiche'], dtype=object),
#   'freq': array([[2, 2],
#          [2, 4],
#          [1, 3]])},
#  {'val': array(['haute', 'normale'], dtype=object),
#   'freq': array([[4, 3],
#          [1, 6]])},
#  {'val': array(['non', 'oui'], dtype=object),
#   'freq': array([[2, 6],
#          [3, 3]])},
#  {'cls': array(['non', 'oui'], dtype=object), 'freq': array([5, 9])}]
#---------------------------------------------------------------------
Theta_jouer = entrainer_multi(X_jouer, Y_jouer)

Theta_jouer

[{'val': array(['ensoleile', 'nuageux', 'pluvieux'], dtype=object),
  'freq': array([[3, 2],
         [0, 4],
         [2, 3]])},
 {'val': array(['chaude', 'douce', 'fraiche'], dtype=object),
  'freq': array([[2, 2],
         [2, 4],
         [1, 3]])},
 {'val': array(['haute', 'normale'], dtype=object),
  'freq': array([[4, 3],
         [1, 6]])},
 {'val': array(['non', 'oui'], dtype=object),
  'freq': array([[2, 6],
         [3, 3]])},
 {'cls': array(['non', 'oui'], dtype=object), 'freq': array([5, 9])}]

### I.4. Estimation de la probabilité de vraissemblance (loi multinomiale)
L'équation pour estimer la vraisemblance 
$$ P(X_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } X_j = v\}|}{|\{y = c_k\}|}$$


Dans le cas d'une valeur $v$ qui n'existe pas dans le dataset d'entrainnement ou qui n'existe pas pour une classe donnée mais ui existe dans le dataset de test, nous aurons une probabilité nulle. 
Afin de régler ce problème, nous pouvons appliquer une fonction de lissage qui attribue une petite probabilité aux données non vues dans l'entraînement. 
Le lissage que nous allons utiliser est celui de Lidstone. 
Lorsque $\alpha = 1$, il est appelé lissage de Laplace.
$$ P(X_j=v|y=c_k) = \frac{|\{ y \in Y / y = c_k \text{ et } X_j = v\}| + \alpha}{|\{y = c_k\}| + \alpha * |V|}$$
Où: 
- $\alpha$ est une valeur donnée 
- $V$ est l'ensemble des différentes valeurs de $f_j$ (le vocabulaire; les catégories)

Etant donné : 
- $\theta_j$ les paramètres de la caractéristique $j$ représentées comme dictionnaire
    - **val** : pour récupérer la liste des noms des catégories de la caractéristique (vocabulaire $V$)
    - **freq**: pour récupérer une matrice représentant la fréquence de chaque caractéristique dans chaque classe. C'est une matrice $|V|\times|C|$
- $v$ la valeur de la caractéristique $j$ utilisée pour calculer les probabilités
- $\theta_c$ les paramètres des classes $C$ représentées comme dictionnaire
    - **cls** : pour récupérer la liste des noms des classes
    - **freq**: pour récupérer la liste des fréquences des classes
    
Cette fonction doit retourner : 
- Une liste $P[|C|]$ contenant les probabilités de la catégorie $v$ de $X_j$ sur toutes les classes $C$ 
- Elle doit prendre en considération le cas où la valeur $v$ n'existe pas dans le modèle entraîné

In [81]:
# TODO: Calculer la vraissamblance d'une valeur donnée
def P_vraiss_multi(Theta_j: Dict[str, np.ndarray], 
                   Theta_c: Dict[str, np.ndarray], 
                   v      : str, 
                   alpha  : float = 0.
                  ) -> np.ndarray: 
    
    # une liste des indices où se trouve la valeur v dans Theta_j["val"]
    # ind est une liste vide s'il n'existe pas un element de Theta_j['val'] egale a v
    # sinon, elle contient au moins une indice (dans notre cas, ecxactement un)
    ind = np.where(Theta_j['val'] == v)[0]
    p = []
    for c in Theta_c['cls']:
        cl_ind = np.where(Theta_c['cls'] == c)[0][0]
        if len(ind) == 0:
            p.append(alpha / (Theta_c['freq'][cl_ind] + alpha * len(Theta_j['val'])))
        else:
            p.append((Theta_j['freq'][ind[0]][cl_ind] + alpha) / (Theta_c['freq'][cl_ind] + alpha * len(Theta_j['val'])))
    return np.array(p)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# (array([0.4       , 0.33333333]), array([0.125     , 0.08333333]))
#---------------------------------------------------------------------
# Calcul :
# La probabilité de jouer si temps = pluvieux 
# P(temps = pluvieux | jouer=oui) = (nbr(temps=pluvieux et jouer=oui)+alpha)/(nbr(jour=oui) + alpha * nbr_diff(temps)))
# P(temps = pluvieux | jouer=oui) = (3 + 0)/(9 + 0) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = pluvieux | jouer=oui) = 4/12 ==> 0.33333333333333333333333333333333333~

# La probabilité de jouer si temps = neigeux 
# P(temps = neigeux | jouer=oui) = (nbr(temps=neigeux et jouer=oui)+alpha)/(nbr(jouer=oui) + alpha * nbr_diff(temps)))
# P(temps = neigeux | jouer=oui) = (0 + 1)/(9 + 3) ==> 3 est le nombre de différentes valeurs de temps (entrainnement)
# P(temps = neigeux | jouer=oui) = 1/13 ==> 0.0833333333333333333333333333333333333~
#---------------------------------------------------------------------

P_vraiss_multi(Theta_jouer[0], Theta_jouer[-1], 'pluvieux'), \
P_vraiss_multi(Theta_jouer[0], Theta_jouer[-1], 'neigeux', alpha=1.)

(array([0.4       , 0.33333333]), array([0.125     , 0.08333333]))

### I.5. Prédiction de la classe (loi multinomiale)
Revenons maintenant à notre équation de prédiction 
$$\hat{c} = \arg\max\limits_{c_k} \log P(y=c_k) + \sum\limits_{f_j \in \overrightarrow{f}} \log P(f_j|y=c_k)$$

- On doit prédire un seule échantillon $x$. 
- La fonction doit retourner un vecteur des log-probabilité des classes
- Si anter=false donc on n'utilise pas la probabilité antérieure

In [82]:
# TODO: Prédiction des log des probabilités
def predire(x    : np.ndarray, 
            Theta: List[Dict[str, np.ndarray]], 
            alpha: float = 1., 
            anter: bool  = True
           ) -> float:
    p = []
    for c in range(len(Theta[-1]['cls'])):
        p.append(np.log(Theta[-1]['freq'][c] / np.sum(Theta[-1]['freq'])) if anter else 0)
        for j in range(len(x)):
            p[-1] += np.log(P_vraiss_multi(Theta[j], Theta[-1], x[j], alpha)[c])
    return np.array(p)

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# (array([-5.20912179, -4.10264337]), array([-4.17950237, -3.66081061]))
#---------------------------------------------------------------------
predire(['pluvieux', 'fraiche', 'normale', 'oui'], Theta_jouer), \
predire(['pluvieux', 'fraiche', 'normale', 'oui'], Theta_jouer, anter=False) 

(array([-5.20912179, -4.10264337]), array([-4.17950237, -3.66081061]))

### I.6. Regrouper en une classe (loi multinomiale)

**Rien à programmer ici**


In [85]:
class NBMultinom(object): 
    
    def __init__(self, alpha=1.): 
        self.alpha = alpha
        
    def entrainer(self, X, Y):
        self.Theta = entrainer_multi(X, Y)
    
    def predire(self, X, anter=True, prob=False): 
        Y_pred = []
        cls = self.Theta[-1]['cls']
        for i in range(len(X)): 
            log_prob = predire(X[i,:], self.Theta, alpha=self.alpha, anter=anter)
            if prob:
                Y_pred.append(np.max(log_prob))
            else:
                Y_pred.append(cls[np.argmax(log_prob)])
        return Y_pred

#=====================================================================
# TEST UNITAIRE
#=====================================================================
# Resultat : 
# ['oui', 'non']
#---------------------------------------------------------------------
notre_modele = NBMultinom()
notre_modele.entrainer(X_jouer, Y_jouer)
X_test = np.array([
    ['neigeux', 'fraiche', 'normale', 'oui'],
    ['neigeux', 'fraiche', 'haute'  , 'oui']
])
notre_modele.predire(X_test)

['oui', 'non']

## II. Application et analyse

**Il n'y a rien à programmer ici.**

Le but de cette section est de mener des expérimentations afin de bien comprendre les concepts vus dans le cours.
Aussi, elle nous assiste à comprendre l'effet des différents paramètres.
En plus, la discussion des différentes expérimentations peut améliorer l'aspect analytique chez l'étudient.

### II.1. Probabilité antérieure 

Nous voulons tester l'effet de la probabilité antérieure.
Pour ce faire, nous avons entraîné deux modèles :
1. Avec probabilité antérieure
1. Sans probabilité antérieure (Il considère une distribution uniforme des classes)

Pour tester si les modèles ont bien s'adapter au dataset d'entraînement, nous allons les tester sur le même dataset et calculer le rapport de classification.


In [89]:
# AVEC Scikit-learn
# ===================
from sklearn.naive_bayes   import CategoricalNB
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics       import classification_report

nb_avec     = CategoricalNB(alpha=1.0, fit_prior=True )
nb_sans     = CategoricalNB(alpha=1.0, fit_prior=False)

enc         = OrdinalEncoder()
X_jouer_enc = enc.fit_transform(X_jouer)
nb_avec.fit(X_jouer_enc, Y_jouer)
nb_sans.fit(X_jouer_enc, Y_jouer)

Y_pred_avec = nb_avec.predict(X_jouer_enc)
Y_pred_sans = nb_sans.predict(X_jouer_enc)

# AVEC notre modèle (juste pour voir comment l'utiliser)
# =======================================================
#notre_modele = NBMultinom()
#notre_modele.entrainer(X_jouer, Y_jouer)
#Y_notre_ant = notre_modele.predire(X_jouer)
#Y_notre_sans_ant = notre_modele.predire(X_jouer, anter=False) 

# Le rapport de classification


print( 'Avec probabilité antérieure (a priori)'  )
print(classification_report(Y_jouer, Y_pred_avec))

print( 'Sans probabilité antérieure (a priori)'  )
print(classification_report(Y_jouer, Y_pred_sans))

Avec probabilité antérieure (a priori)
              precision    recall  f1-score   support

         non       1.00      0.80      0.89         5
         oui       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Sans probabilité antérieure (a priori)
              precision    recall  f1-score   support

         non       0.67      0.80      0.73         5
         oui       0.88      0.78      0.82         9

    accuracy                           0.79        14
   macro avg       0.77      0.79      0.78        14
weighted avg       0.80      0.79      0.79        14



**TODO: Analyser les résultats**
    
- Que remarquez-vous ?
- Est-ce que la probabilité antérieure est importante dans ce cas ?
- Comment cette probabilité affecte le résultat ?
- Quand est-ce que nous sommes sûrs que l'utilisation de cette probabilité est inutile ?

**Réponse**

- ... Le modèle avec probabilité antérieure a un meilleur score que celui sans probabilité antérieure
- ... Oui
- ... Elle augmente la probabilité de la classe la plus probable
- ... Quand les classes sont équilibrées, ce qui veut dire que c'est une probabilité uniforme

### II.2. Lissage

Nous voulons tester l'effet de lissage de Lidstone.
Pour ce faire, nous avons entraîné trois modèles : 
1. alpha = 1 (lissage de Laplace)
1. alpha = 0.5
1. alpha = 0 (sans lissage)

In [92]:
NBC_10 = CategoricalNB(alpha = 1.0 )
NBC_05 = CategoricalNB(alpha = 0.5 )
NBC_00 = CategoricalNB(alpha = 0.0 )

NBC_10.fit( X_jouer_enc,   Y_jouer )
NBC_05.fit( X_jouer_enc,   Y_jouer )
NBC_00.fit( X_jouer_enc,   Y_jouer )

Y_10   = NBC_10.predict(X_jouer_enc)
Y_05   = NBC_05.predict(X_jouer_enc)
Y_00   = NBC_00.predict(X_jouer_enc)


print(          'Alpha = 1.0'             )
print(classification_report(Y_jouer, Y_10))

print(          'Alpha = 0.5'             )
print(classification_report(Y_jouer, Y_05))

print(          'Alpha = 0.0'             )
print(classification_report(Y_jouer, Y_00))


Alpha = 1.0
              precision    recall  f1-score   support

         non       1.00      0.80      0.89         5
         oui       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Alpha = 0.5
              precision    recall  f1-score   support

         non       1.00      0.80      0.89         5
         oui       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93      0.93        14

Alpha = 0.0
              precision    recall  f1-score   support

         non       1.00      0.80      0.89         5
         oui       0.90      1.00      0.95         9

    accuracy                           0.93        14
   macro avg       0.95      0.90      0.92        14
weighted avg       0.94      0.93     

d:\Education\2CS\ML_TPs\venv\Lib\site-packages\sklearn\naive_bayes.py:624: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
d:\Education\2CS\ML_TPs\venv\Lib\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


**TODO: Analyser les résultats**

- Que remarquez-vous ?
- Est-ce que le lissage affecte la performance dans ce cas ? Pourquoi ?
- Pourquoi Scikit-learn n'accepte pas la valeur $\alpha=0$ et affiche une alerte "UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10" ?
- Quelle est l'intérêt du lissage (dans le cas général) ?

**Réponse**

- ... Y a pas de différence entre les trois modèles
- ... Non, car le test est sur le même dataset d'entraînement donc toutes les valeurs de $v$ existent dans le modèle
- ... Car la probabilité des valeurs non vues sera nulle
- ... Pour éviter le sur-apprentissage, car le modèle ne sera pas capable de généraliser en dehor de la dataset d'entraînement

### II.3. Comparaison avec d'autres algorithmes

Naive Bayes est un algorithme puissant lorsqu'il s'agit de classer les documents textuels ; nous voulons tester cette information avec la détection de spam. 
Le dataset utilisé est [SMS Spam Collection Dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset).
Chaque message du dataset doit être représenté sous forme d'un modèle "Sac à mots" (BoW : Bag of Words).
Dans l'entraînement, les différents mots qui s'apparaissent dans les messages (vocabulaire) sont considérés comme des caractéristiques. 
Donc, pour chaque message, la valeur de la caractéristique est la fréquence du mot dans le message. 
Par exemple, si le mot "good" apparait 3 fois dans le message, donc la caractéristique "good" aura la valeur 3 dans ce message.

Notre implémentation n'est pas adéquate pour la nature de ce problème. 
Dans Scikit-learn, [sklearn.naive_bayes.CategoricalNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html) est similaire à notre implémentation. 
L'algorithme adéquat pour ce type de problème est [sklearn.naive_bayes.MultinomialNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).
Les algorithmes comparés :
1. Naive Bayes (Loi Multinomiale)
1. Naive Bayes (Loi Gaussienne)
1. Regression logistique 

In [93]:
# lire le dataset
messages = pd.read_csv('data/spam.csv', encoding='latin-1')
# renomer les caractéristiques : texte et classe
messages = messages.rename(columns={'v1': 'classe', 'v2': 'texte'})
# garder seulement ces deux caractéristiques
messages = messages.filter(['texte', 'classe'])

messages.head()

,texte,classe
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [94]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection         import train_test_split
from sklearn.naive_bayes             import MultinomialNB, GaussianNB
from sklearn.linear_model            import LogisticRegression
from sklearn.metrics                 import precision_score, recall_score
import timeit


modeles = [
    MultinomialNB(),
    GaussianNB(),
    LogisticRegression(solver='lbfgs') 
    #solver=sag est plus lent; donc j'ai choisi le plus rapide
]

temps_train = []
temps_test  = []
rappel      = []
precision   = []

msg_train, msg_test, Y_train, Y_test = train_test_split(messages['texte'] ,
                                                        messages['classe'],
                                                        test_size    = 0.2, 
                                                        random_state = 0  )

count_vectorizer = CountVectorizer()
X_train          = count_vectorizer.fit_transform(msg_train).toarray()
X_test           = count_vectorizer.transform    (msg_test ).toarray()


for modele in modeles:
    # ==================================
    # ENTRAINEMENT 
    # ==================================
    temps_debut = timeit.default_timer()
    modele.fit(X_train, Y_train)
    temps_train.append(timeit.default_timer() - temps_debut)
    
    # ==================================
    # TEST 
    # ==================================
    temps_debut = timeit.default_timer()
    Y_pred      = modele.predict(X_test)
    temps_test.append(timeit.default_timer() - temps_debut)
    
    # ==================================
    # PERFORMANCE 
    # ==================================
    # Ici, nous considérons une classification binaire avec une seule classe "spam" 
    # le classifieur ne sera pas jugé par sa capacité de détecter les non spams
    precision.append(precision_score(Y_test, Y_pred, pos_label='spam'))
    rappel   .append(recall_score   (Y_test, Y_pred, pos_label='spam'))

    
print('Fin')

Fin


#### II.3.1. Temps d'entraînement et de test

Combien de temps chaque algorithme prend pour entrainer le même dataset d'entrainement et combien de temps pour tester le même dataset de test.

In [14]:
algo_noms = ['Naive Bayes Multinomial', 'Naive Bayes Gaussien', 'Regression logistique']

pd.DataFrame({
    'Algorithme'            : algo_noms  ,
    'Temps d\'entrainement' : temps_train,
    'Temps de test'         : temps_test
})

,Algorithme,Temps d'entrainement,Temps de test
0,Naive Bayes Multinomial,0.250683,0.015333
1,Naive Bayes Gaussien,0.303610,0.064949
2,Regression logistique,0.793461,0.017584


**TODO: Analyser les résultats**

- Que remarquez-vous concernant le temps d'entrainement ? (ordonner les algorithmes)
- Pourquoi nous avons eu ces résultats en se basant sur les algorithmes ? (discuter chaque algorithme vis-a-vis le temps d'entrainement)
- Que remarquez-vous concernant le temps de test ? (ordonner les algorithmes)
- Pourquoi nous avons eu ces résultats en se basant sur les algorithmes ? (discuter chaque algorithme vis-a-vis le temps de test)

**Réponse**

- ... Le temps d'entraînement est le plus petit pour la régression logistique, puis Naive Bayes (loi multinomiale) et enfin Naive Bayes (loi gaussienne)
- ... La régression logistique est plus lente car elle est une méthode d'optimisation itérative alors que les méthodes Naive Bayes sont des solutions directes, et Naive Bayes (loi gaussienne) est plus lente que Naive Bayes (loi multinomiale) car elle calcule la moyenne et la variance de chaque caractéristique.
- ... Le temps de test est le plus petit pour Naive Bayes (loi multinomiale), puis la régression logistique et enfin Naive Bayes (loi gaussienne).
- ... Naive Bayes (loi multinomiale) est plus rapide que les autres car elle ne calcule que la probabilité de chaque classe, et la Regression Logistique est plus rapide que Naive Bayes (loi gaussienne) car elle ne calcule que le produit scalaire entre les caractéristiques et les poids, alors que Naive Bayes (loi gaussienne) calcule la densité de probabilité gaussienne pour chaque caractéristique.

#### II.3.2. Qualité de prédiction

Comment chaque algorithme performe sur le dataset de test dans le cas de détection de spams (spam: est la classe positive).

In [15]:
pd.DataFrame({
    'Algorithme' : algo_noms,
    'Rappel'     : rappel   ,
    'Precision'  : precision
})

,Algorithme,Rappel,Precision
0,Naive Bayes Multinomial,0.927711,0.987179
1,Naive Bayes Gaussien,0.891566,0.616667
2,Regression logistique,0.855422,0.986111


**TODO: Analyser les résultats**

On remarque que Naive Bayes surpasse la régression logistique pour la détection de spams. 
- Est-ce que ceci preuve que Naive Bayes est meilleur que les autres algorithmes sur n'importe quel problème ?
- Est-ce que ceci preuve que Naive Bayes peut donner de meilleurs résultats que les autres algorithmes sur des problèmes similaires ?
- Pourquoi le modèle gaussien est moins performant que le multinomial en se basant sur la nature des deux algorithmes ?
- Pourquoi le modèle gaussien est moins performant que le multinomial en se basant sur la nature du probleme/donnees ?

**Réponse**

- Non
- Oui
- ... Car la méthode gaussienne stipule que les caractéristiques suivent une distribution gaussienne, ce qui n'est pas forcément le cas dans ce problème
- ... Car les caractéristiques sont des mots, et la distribution gaussienne n'est pas adaptée pour les mots (les mots sont des valeurs discrètes)

In [101]:
print("  _____    __                                              _               ")
print(" |_   _|  / _|                                            | |              ")
print("   | |   | |_     _   _    ___    _   _      __ _    ___  | |_             ")
print("   | |   |  _|   | | | |  / _ \  | | | |    / _` |  / _ \ | __|            ")
print("  _| |_  | |     | |_| | | (_) | | |_| |   | (_| | |  __/ | |_             ")
print(" |_____| |_|      \__, |  \___/   \__,_|    \__, |  \___|  \__|            ")
print("                   __/ |                     __/ |                         ")
print("                  |___/                     |___/                          ")
print("  _     _       _            __                                            ")
print(" | |   | |     (_)          / _|                 _                         ")
print(" | |_  | |__    _   ___    | |_    __ _   _ __  (_)                        ")
print(" | __| | '_ \  | | / __|   |  _|  / _` | | '__|                            ")
print(" | |_  | | | | | | \__ \   | |   | (_| | | |     _                         ")
print("  \__| |_| |_| |_| |___/   |_|    \__,_| |_|    ( )                        ")
print("                                                |/                         ")
print("                                                                           ")
print("                                                                           ")
print("                                                                           ")
print("  _   _    ___    _   _      __ _   _ __    ___                            ")
print(" | | | |  / _ \  | | | |    / _` | | '__|  / _ \                           ")
print(" | |_| | | (_) | | |_| |   | (_| | | |    |  __/                           ")
print("  \__, |  \___/   \__,_|    \__,_| |_|     \___|                           ")
print("   __/ |                                                                   ")
print("  |___/                                                                    ")
print("                    _                                                __    ")
print("                   | |                                            _  \ \   ")
print("  _ __     ___     | |__    _   _   _ __ ___     __ _   _ __     (_)  | |  ")
print(" | '_ \   / _ \    | '_ \  | | | | | '_ ` _ \   / _` | | '_ \         | |  ")
print(" | | | | | (_) |   | | | | | |_| | | | | | | | | (_| | | | | |    _   | |  ")
print(" |_| |_|  \___/    |_| |_|  \__,_| |_| |_| |_|  \__,_| |_| |_|   (_)  | |  ")
print("                                                                     /_/   ")
print("                                                                           ")

  _____    __                                              _               
 |_   _|  / _|                                            | |              
   | |   | |_     _   _    ___    _   _      __ _    ___  | |_             
   | |   |  _|   | | | |  / _ \  | | | |    / _` |  / _ \ | __|            
  _| |_  | |     | |_| | | (_) | | |_| |   | (_| | |  __/ | |_             
 |_____| |_|      \__, |  \___/   \__,_|    \__, |  \___|  \__|            
                   __/ |                     __/ |                         
                  |___/                     |___/                          
  _     _       _            __                                            
 | |   | |     (_)          / _|                 _                         
 | |_  | |__    _   ___    | |_    __ _   _ __  (_)                        
 | __| | '_ \  | | / __|   |  _|  / _` | | '__|                            
 | |_  | | | | | | \__ \   | |   | (_| | | |     _                         
  \__| |_| |

<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:6: SyntaxWarning: invalid escape sequence '\_'
<>:12: SyntaxWarning: invalid escape sequence '\ '
<>:13: SyntaxWarning: invalid escape sequence '\_'
<>:14: SyntaxWarning: invalid escape sequence '\_'
<>:20: SyntaxWarning: invalid escape sequence '\ '
<>:22: SyntaxWarning: invalid escape sequence '\_'
<>:26: SyntaxWarning: invalid escape sequence '\ '
<>:28: SyntaxWarning: invalid escape sequence '\ '
<>:30: SyntaxWarning: invalid escape sequence '\_'
<>:4: SyntaxWarning: invalid escape sequence '\ '
<>:6: SyntaxWarning: invalid escape sequence '\_'
<>:12: SyntaxWarning: invalid escape sequence '\ '
<>:13: SyntaxWarning: invalid escape sequence '\_'
<>:14: SyntaxWarning: invalid escape sequence '\_'
<>:20: SyntaxWarning: invalid escape sequence '\ '
<>:22: SyntaxWarning: invalid escape sequence '\_'
<>:26: SyntaxWarning: invalid escape sequence '\ '
<>:28: SyntaxWarning: invalid escape sequence '\ '
<>:30: SyntaxWarning: invalid escap